In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000905_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2020-10-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

hs300cs = trdb2py.trading2_pb2.Candle(
    ts=int(trdb2py.str2timestamp('2021-03-12', '%Y-%m-%d')),
    open=51418700,
    close=51418700,
    high=51418700,
    low=51418700,    
)

hs300 = trdb2py.trading2_pb2.Candles(
    market='jqdata',
    symbol='000905_XSHG|1d',
    candles=[],
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='中证500',
    offset=28,
    candles=[hs300],
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0, ignoreCache=True)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

我们还是拿沪深300指数，2013年5月1日到2020年9月30日的日线数据来做例子。

In [ ]:
ret = trdb2py.getAssetCandles2(trdb2cfg, asset, tsStart, tsEnd, indicators=['ta-sma.17', 'ta-sma.28'], offset=28, simCandle=hs300, indicatorScale=10000)
# print(ret) 
# ret
trdb2py.showAssetCandles2('中证500指数', ret, indicators=['ta-sma.17', 'ta-sma.28'], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstems = [42]
# 起始时间，0表示从最开始算起
# tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

lstema0 = [17]
lstema1 = [28]


for i in range(0, len(lstema0)):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
    )
    
    buy21 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatordp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema0[i]), 'ta-sma.{}'.format(lstema1[i])],
    )    

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
    )        
    
    buy4 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,
    )

    buy5 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
        group=1,        
    )

    buy6 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
        group=1,        
    )
    
    buy61 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatordp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema0[i]), 'ta-sma.{}'.format(lstema1[i])],
        group=1,        
    )    

    buy7 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
        group=1,        
    )            

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )
    
    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,                
    )    

#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

#     s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6])
    s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6, buy7])
    s0.sell.extend([sell0, sell1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma{}&sma{}'.format(lstema0[i], lstema1[i]),
        offset=lstema1[i],
        candles=[hs300],        
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3, ignoreCache=True)

# trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)
sts = trdb2py.getFirstCtrlTs(lstpnlmix[0])

trdb2py.showPNLs([lstpnlmix[0], pnlBaseline], isShowBuy=True, isShowSell=True, toImg=isStaticImg, width=width, height=height, startTs=sts)

In [ ]:
npnl0 = trdb2py.clonePNLWithTs(pnlBaseline['pnl'], sts)
npnl1 = trdb2py.clonePNLWithTs(lstpnlmix[0]['pnl'], sts)

lstallpnl = [{'title': '中证500', 'pnl': npnl0},
             {'title': '17&28日均线', 'pnl': npnl1}]

for v in lstallpnl:
    trdb2py.rebuildPNL(v['pnl'])
    
dflstallpnl = trdb2py.buildPNLReport(lstallpnl)

dflstallpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility']].sort_values(by='totalReturns', ascending=False)
# dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

# # dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

# dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)